#### Create initial dataframe

In [ ]:
# Create initial dataframe
import pandas as pd

file = pd.read_csv('../transitions/transition_C04B2C5EEB24_1503931079.181505_0_6.csv',  sep=' ', header = None) 

df = pd.DataFrame(file)
df.columns = ['mag_x', 'mag_y', 'mag_z', 'temp', 'acc_x', 'acc_y', 'acc_z', 'gyr_x', 'gyr_y', 'gyr_z', 'vib_ind', 'ts']
df.head()


#### Run calculations

In [ ]:
from prop_defs import transform_units, angles_comp, kalman_filter, mag_yaw

def run_calcs(df):

    # Apply calibrations and sensitivity
    pdf = transform_units(df, 1,1,1) # acc, gyr, mag calibrated

    # Compute euler angles
    cdf = angles_comp(pdf, 50) # 50 Hz
    cdf['ts'] = cdf.index

    # Kalman filter
    kdf = kalman_filter(cdf, 1, 50) # vibration on, 50 Hz

    # Tilt-compensated yaw computation with magnetometer
    vals_df = mag_yaw(kdf, pdf) # kalman results, data with magnetometer values
    vals_df['ts'] = vals_df.index # time index

    # View dataframe
    vals_df = pd.merge(vals_df, cdf[['acc_angle_z', 'ts']], how = 'inner', on = 'ts') # Optional step, add tilt angle
    
    return vals_df

In [ ]:
%time vals_df = run_calcs(df)
vals_df.shape

In [ ]:
vals_df.tail()

In [ ]:
import matplotlib.pyplot as plt

df = vals_df

# Optional conversion
dt = 0.02 # 1/Sampling rate Hz
ts = df.ts * dt

# Plots

fig, ax = plt.subplots()

ax.plot(ts, df['kalman_roll'], color='r', label='kalman_roll')
ax.plot(ts, df['kalman_pitch'], color='b', label='kalman_pitch')
ax.plot(ts, df['acc_angle_z'], color='y', label='tilt angle - acc')
ax.plot(ts, df['yaw'], color='g', label='yaw')

handles, labels = ax.get_legend_handles_labels()
ax.legend(handles, labels, bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
ax.set_xlabel('Time (secs)')
ax.set_ylabel('$^\circ$ degrees')
plt.title('Rotation Angles')
plt.show(fig)
